# Make Your Own Reduction Scrypt (photometry)

*Work in Progress*

ASTROPOP is not a reduction script by itself, but a library containing (almost) everything you need to create your reduction script by yourself.

In this guide we will follow a standard reduction procedure using the ASTROPOP modules and you can follow it to perform your own reduction.

If you want to use this notebook directly in your Jupyter, you can download it in [this link](https://raw.githubusercontent.com/sparc4-dev/astropop/main/docs/ipynb/diy_reduction_script.ipynb).

## Organize Your Data

The first step is to organize your data. *Do not forget to backup all the raw data before start your reduction!*

Here we will perform a reduction using just bias and flatfield corrections, in a single band. No dark frame subtraction will be used, so we do not have to set it.

First, we will have to create the lists containing the image names we need. You can do it manually, using [glob](https://docs.python.org/3/library/glob.html) package, or the built-in file organizer of ASTROPOP. This last option allow filtering using header keys, but may be slow and memory consuming for very large sets (with thousands of images). For didatic reasons, we will to it with ASTROPOP file organizer.

Is convenient to set now some directory names for raw, reduced and temporary files, as in the code bellow.

In [ ]:
import os
# Directories where we will storefiles
base_dir = os.path.expanduser('~/astropop-tutorial/photometry-example/raw')
raw_dir = os.path.join(base_dir, 'raw')  # Directory containing the rawdata
reduced_dir = os.path.join(base_dir, 'reduced')  # Directory to store processed data
tmp_dir = os.path.join(base_dir, 'tmp')  # Directory to store temporary files

# create the dir if not exists
os.makedirs(raw_dir, exist_ok=True)  
os.makedirs(reduced_dir, exist_ok=True)
os.makedirs(tmp_dir, exist_ok=True)

In this tutorial, we will a set of images of HD5980 star taken in Observatório do Pico dos Dias (OPD/LNA) and available in [github/sparc4-dev/astropop-data](https://github.com/sparc4-dev/astropop-data) repository.

To download these data in the raw folder, follow the script bellow.

In [ ]:
# Downloading the data
import urllib.request
import os

with urllib.request.urlopen('https://raw.githubusercontent.com/sparc4-dev/astropop-data/main/raw_images/opd_ixon_bc_hd5980/filelist.txt') as f:
    for line in f.readlines():
        url = line.decode('UTF-8')
        filename = os.path.join(raw_dir, url.split('/')[-1].split('?')[0])
        urllib.request.urlretrieve(url, filename)

With `astropop.file_collection.FitsFileGroup` we will create a dynamic library of fits files. This library allow iterate over its files, filter by header keywords, access file names or direct access a given index. In this case, we will access only the file names via `FitsFileGroup.files` property.

In [ ]:
from astropop.file_collection import FitsFileGroup

main_fg = FitsFileGroup(location=raw_dir, fits_ext=['.fits'])

print(f'Total files: {len(main_fg)}')

# Filter files by header keywords
bias_fg = main_fg.filtered({'obstype': 'BIAS'})  # OBSTYPE='BIAS'
print(f'Bias files: {len(bias_fg)}')
print(bias_fg.files)
flat_fg = main_fg.filtered({'obstype': 'FLAT'})  # OBSTYPE='FLAT'
print(f'Flat files: {len(flat_fg)}')
print(flat_fg.files)
star_fg = main_fg.filtered({'obstype': 'SCIENCE', 'object': 'HD5980'})  # OBSTYPE='SCIENCE' and OBJECT='HD5980'
print(f'Star files: {len(star_fg)}')
print(star_fg.files)

## Master Calibration Frames

Before process the science images, we need to create the `master_bias` and `master_flat` calibration frames.

### Master Bias

## Basic CCD Image Processing 

## Align and Stack

## Source Detection and Photometry

## Photometry Calibration with Online Catalogs